In [1]:
print('unziping ...')
!unzip -o -j sms+spam+collection.zip 

unziping ...
Archive:  sms+spam+collection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [51]:
import pandas as pd
messages=pd.read_csv('SMSSpamCollection', sep='\t', names=['label','message'])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [52]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
messages['label']=le.fit_transform(messages['label'])
mapping = dict(zip(le.classes_, range(0, len(le.classes_)+1)))
print(mapping)
messages.head()

{'ham': 0, 'spam': 1}


,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [53]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [54]:
# Data cleaning and preprocessing
corpus=[]
lem=WordNetLemmatizer()
for i in range(len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['message'][i])
    review=review.lower().split()
    review=[lem.lemmatize(word) for word in review if not word in set[stopwords.words('english')]]
    review=' '.join(review)
    corpus.append(review)

In [55]:
# TF-IDF model creation
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=2500)
X=cv.fit_transform(corpus).toarray()


In [56]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, messages['label'], test_size=0.20, random_state=0) 

In [57]:
# Training model using Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train,Y_train) 

Y_pred = spam_detect_model.predict(X_test)

In [58]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(Y_test,Y_pred)

array([[955,   0],
       [ 25, 135]])

In [59]:
# Checking accuracy of model
print("Accuracy of model: ",accuracy_score(Y_test, Y_pred))

Accuracy of model:  0.9775784753363229
